In [492]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

We use the following conventions throughout. We use $x \in [0,1]$ as the time variable. For $i = 0,1, \dots, d$, $y_{i}(x)$ is the proportion of vertices of degree $i$ at time $x$, and we let $y(x) = (y_{1}(x),\dots,y_{d}(x))$.

Utility functions.

In [496]:
def delta(a,b):#dirac delta function
    if a == b:
        return 1
    else:
        return 0

def S(y, d):#returns the total degree of the remaining graph at time t
    return sum([(i+1)*y[i] for i in range(d)])

def f(x, y, i, r, d):#returns (approximate) expected change in Y_{i} conditioned on the history up to time t and on processing a vertex of degree r at time t
    total_degree = S(y, d)
    term1 = -delta(i,r) - (r*i*y[i-1])/total_degree
    if i < d:
        return term1 + (r*(i+1)*y[i])/total_degree
    else:
        return term1

#beta and tau are used define the proportion of each selection type in phase k for the deprioritized algorithm
def beta(x, y, k, d):
    return f(x, y, d-k-1, d-k, d)
    
def tau(x, y, k, d):
    return -f(x, y, d-k-1, d-k-1, d)

Defines the system of ODEs as a function of $x$, $y$, and $k$.

In [499]:
def degree_evolution(x, y, k, d):#returns 1dim array of length d which gives RHS of the ODE system for phase k, k = 1,2,...,d-1
    F = [0]*d
    A = tau(x, y, k, d)/(beta(x, y, k, d) + tau(x, y, k, d))
    B = beta(x, y, k, d)/(beta(x, y, k, d) + tau(x, y, k, d))
    if k <= d-2:
        for i in range(d):
            F[i] = A*f(x, y, i+1, d-k, d) + B*f(x, y, i+1, d-k-1, d)
    elif k == d-1:
        for i in range(d):
            F[i] = f(x, y, i+1, 1, d)
    return F

Event functions: for each function $G$ below, the solver records the times $x$ for which $G(x) = 0$.

In [502]:
def critical(x, y, k, d):#returns y_{d}' - (-1), the solution of which is the critical point of the system
    return degree_evolution(x, y, k, d)[d-1] + 1

def phase_terminate(x, y, k, d):#used for termination condition for phase k: tau = 0
    return tau(x, y, k, d)
phase_terminate.terminal = True #ensures the integration halts if we reach a state with tau = 0

The solver.

In [505]:
def mingreedySolve(d):#returns a 1dim array of scipy ODEsolution objects, where the entry at index k-1 is the solution during phase k
    y0 = [0]*(d-1) + [1]
    x0 = 0
    sols = []
    for k in range(1,d):
        sol = solve_ivp(degree_evolution, [x0, 1], y0, dense_output = True, events = (phase_terminate, critical), args = (k,d))
        sols.append(sol)
        if sol.status == 0:#sol.status = 0 iff it reaches the end of the integration interval, i.e, x = 1
            break
        else:
            x0 = sol.t_events[0][0]
            y0 = sol.sol(x0)
    return sols

Solves the IVP for all $d$ in a given range, and computes $\sum_{i=0}^{d-1}y_{i}(x^{*})$, where $x^{*}$ is such that $y_{d}'(x^{*}) + 1 = 0$.

In [524]:
upper_bounds = []
for d in range(3, 31):
    sols = mingreedySolve(d)
    for sol in sols:
        if sol.t_events[1].size > 0 and sol.t_events[1][0] <= 0.9999:#second condition is there in order to ignore superfluous t_events at x = 1
            M = sol.y_events[1][0]
            upper_bounds.append((d, M[0:d-1].sum()))

print(f"{"degree (d)":<20} bound")
print("-"*28)
for d, y in upper_bounds:
    print(f"{d:>2} {y:>24,}")

degree (d)           bound
----------------------------
 3      0.22218760020623377
 4      0.33305439527567027
 5      0.42390737787442445
 6       0.4910418721601975
 7       0.5400987998646845
 8       0.5816588940036727
 9       0.6162415366471109
10       0.6445529549683559
11       0.6681234793941936
12       0.6888537944875079
13       0.7071825016068469
14        0.723273032221251
15       0.7374538072196422
16       0.7500199628381669
17       0.7612669072510119
18       0.7715549874784335
19       0.7809761848225099
20        0.789603802248852
21       0.7975150303584235
22        0.804784976629285
23       0.8114777807697199
24       0.8176455445357618
25       0.8233881162861584
26       0.8287717076068477
27       0.8338208838881924
28       0.8385620964725898
29       0.8430250517542714
30       0.8472315086096358
